Author:
        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/4956778/8344638/a2a6aa289fce8461958dc287f1dab799/dataset-cover.jpg?t=2024-05-07-09-36-53" alt="IMG">
</p>

<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 1. Content</i></h1>

<br>

> <h4 style = 'font-family: Times New Roman'>
This dataset explores how weather conditions impact renewable energy generation. <br><br>Spanning from January 1, 2017, to August 31, 2022, the dataset provides climate data such as temperature, pressure, wind speed, and sunlight duration at 15-minute intervals. <br><br>By incorporating variables like GHI and SunlightTime, it enables the prediction of solar energy production.


- DataSource: https://www.kaggle.com/datasets/pythonafroz/renewable-power-generation-and-weather-conditions/data
  
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 2. About Dataset</i></h1>

<br> 

<details>
    <summary>More Columns Info</summary>
    17 columns in the dataset, below is <u>description of main features</u>:

    *  (1) Time: The timestamp of the recorded data in the format of YYYY-MM-DD HH:MM:SS.
        
    *  (2) Energy delta[Wh]: The difference in energy consumption in Watt-hours (Wh) from the previous timestamp to the current timestamp.
        
    *  (3) GHI: Global Horizontal Irradiance in Watts per square meter (W/m²) measured by a pyranometer.
        
    *  (4) temp: The temperature in degrees Celsius (°C) measured at the same height as the pyranometer.

    *  (5) pressure: The atmospheric pressure in hectopascals (hPa) measured at the same height as the pyranometer.

    *  (6) humidity: The relative humidity in percentage (%) measured at the same height as the pyranometer.

    *  (7) wind_speed: The wind speed in meters per second (m/s) measured at the same height as the pyranometer.

    *  (8) rain_1h: The amount of precipitation in millimeters (mm) measured over the past hour.
    
    *  (9) snow_1h: The amount of snowfall in millimeters.

    *  (10) clouds_all: The cloud situation.
</details>    
    
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 3. Goal of the Notebook</i></h1>
    
> <h4 style = 'font-family: Times New Roman'>
Goal is to train and test model using GPT, Pytorch. <br><br>
    The Target Column used here is <b>Energy delta[Wh]</b>
</h4>

 

***

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import pandas  as pd
df = pd.read_csv(path_append + '../data/renewable power generation/Renewable.csv')
df.head()

,Time,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month
0,2017-01-01 00:00:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
1,2017-01-01 00:15:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
2,2017-01-01 00:30:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
3,2017-01-01 00:45:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
4,2017-01-01 01:00:00,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0,4,1,1


<a id="2"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #4290f5'>2.</b> Modeling: Preprocess </b></h1>

In [4]:
# Remove rows where energy delta is 0
df1 = df[df['Energy delta[Wh]'] != 0] 
df1.head()

,Time,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month
32,2017-01-01 08:00:00,5,10.6,3.5,1016,99,6.0,0.0,0.0,98,1,60,450,0.13,4,8,1
33,2017-01-01 08:15:00,33,6.0,3.5,1016,99,6.0,0.0,0.0,98,1,75,450,0.17,4,8,1
34,2017-01-01 08:30:00,44,2.8,3.5,1016,99,6.0,0.0,0.0,98,1,90,450,0.20,4,8,1
35,2017-01-01 08:45:00,61,3.1,3.5,1016,99,6.0,0.0,0.0,98,1,105,450,0.23,4,8,1
36,2017-01-01 09:00:00,65,3.5,3.6,1016,97,6.2,0.0,0.0,100,1,120,450,0.27,4,9,1


In [5]:
df1.tail()

,Time,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month
196770,2022-08-31 16:30:00,157,31.9,18.6,1023,57,3.8,0.0,0.0,52,1,765,825,0.93,3,16,8
196771,2022-08-31 16:45:00,118,23.7,18.6,1023,57,3.8,0.0,0.0,52,1,780,825,0.95,3,16,8
196772,2022-08-31 17:00:00,82,15.6,18.5,1023,61,4.2,0.0,0.0,60,1,795,825,0.96,3,17,8
196773,2022-08-31 17:15:00,51,8.0,18.5,1023,61,4.2,0.0,0.0,60,1,810,825,0.98,3,17,8
196774,2022-08-31 17:30:00,24,2.1,18.5,1023,61,4.2,0.0,0.0,60,1,825,825,1.00,3,17,8


In [6]:
# Convert Time to datetime and then to timestamp
df1['Time'] = pd.to_datetime(df1['Time'], format='%Y-%m-%d %H:%M:%S')

df1['Timestamp'] = df1['Time'].astype('int64') // 10**9  # Convert to seconds

df1.drop(['Time'], axis=1, inplace=True)

df1.head()

,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month,Timestamp
32,5,10.6,3.5,1016,99,6.0,0.0,0.0,98,1,60,450,0.13,4,8,1,1483257600
33,33,6.0,3.5,1016,99,6.0,0.0,0.0,98,1,75,450,0.17,4,8,1,1483258500
34,44,2.8,3.5,1016,99,6.0,0.0,0.0,98,1,90,450,0.20,4,8,1,1483259400
35,61,3.1,3.5,1016,99,6.0,0.0,0.0,98,1,105,450,0.23,4,8,1,1483260300
36,65,3.5,3.6,1016,97,6.2,0.0,0.0,100,1,120,450,0.27,4,9,1,1483261200


In [7]:
from sklearn.model_selection import train_test_split

X = df1.drop(columns=['Energy delta[Wh]'])
y = df1['Energy delta[Wh]'] 

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Concatenate the X_train, y_train, X_test, y_test to create train and test dataframes
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [8]:
train_df.head()

,GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month,Timestamp,Energy delta[Wh]
187938,59.7,17.4,1014,53,1.0,0.0,0.0,54,1,840,990,0.85,3,16,5,1654014600,490
38076,15.0,-2.0,1019,85,1.4,0.0,0.0,94,1,525,555,0.95,4,15,2,1517929200,254
77498,22.1,2.2,1028,90,2.6,0.0,0.0,99,1,120,750,0.16,4,6,3,1553409000,293
185545,1.9,15.6,1023,66,1.2,0.0,0.0,56,1,915,930,0.98,3,18,5,1651860900,19
89885,41.4,16.4,1016,97,2.1,0.0,0.0,36,1,255,945,0.27,2,7,8,1564643700,857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13356,3.2,6.7,1019,94,3.3,0.0,0.0,14,1,30,990,0.03,2,3,5,1495681200,32
115796,37.7,1.3,1010,93,1.0,0.0,0.0,0,1,105,900,0.12,1,5,4,1587963600,290
158470,34.6,25.2,1012,78,2.4,0.0,0.0,73,1,900,990,0.91,3,17,7,1626370200,161
2727,7.6,-0.6,1010,93,4.2,0.0,0.0,96,1,195,540,0.36,4,9,2,1486115100,520


In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Assuming train_df and test_df are pandas DataFrames

# Initialize scalers
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler(feature_range = (-1, 1))

# Apply StandardScaler to all but the last column
train_df.iloc[:, :-1] = standard_scaler.fit_transform(train_df.iloc[:, :-1].astype(float))
test_df.iloc[:, :-1] = standard_scaler.transform(test_df.iloc[:, :-1].astype(float))

# Ensure the last column is converted to float
train_df.iloc[:, -1] = train_df.iloc[:, -1].astype(float)
test_df.iloc[:, -1] = test_df.iloc[:, -1].astype(float)

# Apply MinMaxScaler to the last column
train_df.iloc[:, -1] =  min_max_scaler.fit_transform(train_df.iloc[:, -1].values.reshape(-1, 1))
test_df.iloc[:, -1] = min_max_scaler.transform(test_df.iloc[:, -1].values.reshape(-1, 1))

# Verification of the number of features
num_features = train_df.shape[1]


In [11]:
import torch
from torch.utils.data import Dataset
import random

class SequentialDataset(Dataset):
    def __init__(self, df, indices, max_window_size, num_classes):
        self.df = df
        self.indices = indices
        self.max_window_size = max_window_size
        self.min_window_size = max_window_size // 2
        self.num_classes = num_classes

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        start_idx = self.indices[idx]
        window_size = random.randint(self.min_window_size, self.max_window_size)
        end_idx = min(start_idx + window_size, len(self.df))

        seq = self.df.iloc[start_idx:end_idx]
        X, y = seq.values[:-1, :], seq.values[1:, :1] # mean temperature is the last column

        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return X, y

In [12]:
from random import shuffle

# Assume 'df' is your DataFrame and 'event' is the column containing labels

def generate_indices(df, max_window_size):
    indices = []
    max_index = len(df) - max_window_size + 1  # Calculate the maximum starting index
    
    for i in range(max_index):
        # Check if all labels in the window are the same
        indices.append(i)
    
    return indices

# Assuming 'df' and 'num_classes' are defined
max_window_size = 14
train_indices = generate_indices(train_df, max_window_size)
test_indices = generate_indices(test_df, max_window_size)

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
print('Train indices: ', len(train_indices))
print('Test indices: ', len(test_indices))

trainset = SequentialDataset(df=train_df, indices=train_indices, max_window_size=max_window_size, num_classes=num_features)
testset = SequentialDataset(df=test_df, indices=test_indices, max_window_size=max_window_size, num_classes=num_features)

train df shape:  (76740, 17)
test df shape:  (19186, 17)
Train indices:  76727
Test indices:  19173


<a id="3"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #427bf5'>3.</b> Modeling</b></h1>

In [13]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

data_config = DataConfig(dataset_name = 'renewable-power-gen-prediction', task_type='regression', obs_shape=[num_features], label_size=1)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(core_model = 'gpt', encoder_model = 'none')

ml_params.num_epoch = 1000
ml_params.model.core_config.dropout = 0.1
ml_params.optimization.decay_rate_100k = 0.001

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False) 

In [14]:
trainer_hub.train(trainset, testset)

Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

[0/1000][50/1198][Time 9.22]
Unified LR across all optimizers: 0.0001992966486266836
--------------------Training Metrics--------------------
Trainer:  gpt
Inf: 0.1476	Gen: 0.3361	Rec: 0.3202	E: 0.1629	R: 0.1303	P: 0.5074
--------------------Test Metrics------------------------
mse: 25.7548
mae: 4.6083
r2: -28.0906

[0/1000][100/1198][Time 7.54]
Unified LR across all optimizers: 0.00019860948975280947
--------------------Training Metrics--------------------
Trainer:  gpt
Inf: 0.0467	Gen: 0.1420	Rec: 0.1377	E: 0.0498	R: 0.0412	P: 0.2337
--------------------Test Metrics------------------------
mse: 63.8385
mae: 7.8485
r2: -73.7474

[0/1000][150/1198][Time 7.70]
Unified LR across all optimizers: 0.00019792470014766712
--------------------Training Metrics--------------------
Trainer:  gpt
Inf: 0.0319	Gen: 0.1158	Rec: 0.1134	E: 0.0337	R: 0.0291	P: 0.1971
--------------------Test Metrics------------------------
mse: 78.4195
mae: 8.7333
r2: -58.3679

[0/1000][200/1198][Time 7.49]
Unified LR a